In [ ]:
import requests

url = "https://www.iiserbpr.ac.in/people/faculty"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers, verify=False)  # disable SSL verification
html = response.text

print(html[:500])


/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iiserbpr.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta name="google-site-verification" content="JIn0JQlmj5Z1k5AUpZnb6AQbee1qRQSFiEKN2YfrPhg" />
  <title>People</title>
  <link rel="stylesheet" href="https://www.iiserbpr.ac.in/assets/css/bootstrap.min.css">
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css" />

  <link rel="stylesheet" href="


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

BASE_URL = "https://www.iiserbpr.ac.in"
FACULTY_PAGE = f"{BASE_URL}/people/faculty"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(FACULTY_PAGE, headers=headers, verify=False)
response.raise_for_status()
soup = BeautifulSoup(response.text, "html.parser")

faculty_blocks = soup.select("div.card.boxsections")

with open("iiserbpr_faculty.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Name", "Designation", "Email", "Profile Link", "PhD", "Postdoc"])

    for block in faculty_blocks:
        name_tag = block.select_one("div.offcheading")
        name = name_tag.get_text(strip=True) if name_tag else ""

        designation = ""
        email = ""
        for li in block.select("ul.contact-info li"):
            text = li.get_text(strip=True)
            if "@" in text:
                email = text
            elif "Professor" in text or "Director" in text:
                designation = text

        a_tag = block.select_one("a.btn.btn-primary")
        rel_link = a_tag.get("href") if a_tag else ""
        profile_link = BASE_URL + rel_link if rel_link.startswith("/") else rel_link

        phd = ""
        postdoc = ""

        # ---- Scrape individual faculty profile ----
        if profile_link:
            try:
                prof_resp = requests.get(profile_link, headers=headers, verify=False)
                prof_resp.raise_for_status()
                prof_soup = BeautifulSoup(prof_resp.text, "html.parser")

                # The “Academic Background” section has headings and paragraphs/lists
                sections = prof_soup.select("div.col-md-12 div.row.clearfix")
                for sec in sections:
                    heading = sec.select_one("h3, h4, h5")
                    heading_text = heading.get_text(strip=True).lower() if heading else ""

                    # Target “academic background” or “education” sections
                    if "academic" in heading_text or "education" in heading_text:
                        content = sec.get_text(separator="\n", strip=True)
                        lines = [l.strip() for l in content.split("\n") if l.strip()]
                        for line in lines:
                            for line in text.split(","):
                             if "Ph.D." in line or "PhD" in line:
                                 phd = line.strip()
                             if "Postdoc" in line or "Postdoctoral" in line:
                                 postdoc_info = line.strip()

                # Fallback if data not inside 'academic' div
                if not phd or not postdoc:
                    all_text = prof_soup.get_text(separator="\n")
                    for line in all_text.split("\n"):
                        line = line.strip()
                        if not phd and ("Ph.D." in line or "PhD" in line):
                            phd = line
                        if not postdoc and ("Postdoctoral" in line or "Postdoc" in line):
                            postdoc = line

                time.sleep(0.3)

            except Exception as e:
                print(f"⚠️ Failed to fetch {profile_link}: {e}")

        writer.writerow([name, designation, email, profile_link, phd, postdoc])

print("✅ Scraping completed! Data saved to iiserbpr_faculty.csv")


/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iiserbpr.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iiserbpr.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.iiserbpr.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/codespace/.local

✅ Scraping completed! Data saved to iiserbpr_faculty.csv
